In [93]:
import requests
import pandas as pd
import datetime
import config

In [94]:
# Установим параметры для запроса к API Binance
symbol = 'BTCUSDT'  # Пара торгов BTCUSDT (для биткоина и доллара США)
interval = '1d'     # Интервал времени - 5m

def get_data(start_time):
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': 1000,  # Лимит на количество записей, возвращаемых API Binance, составляет 1000
        'startTime': start_time,
    }
    # Выполним GET-запрос к API Binance
    url = 'https://api.binance.com/api/v3/klines'
    response = requests.get(url, params=params)
    # Проверяем успешность запроса
    if response.status_code == 200:
        data = response.json()
        # Конвертация json в DataFrame с соответствующими столбцами
        df = pd.DataFrame(data, columns=['open_time', 'open', 'high', 'low', 'close', 'volume',
                                          'close_time', 'quote_asset_volume', 'number_of_trades',
                                          'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume',
                                          'ignore'])
        # Переведем столбец даты и времени в формат datetime64[ns]
        df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
        df.rename(columns={'open_time': 'datetime'}, inplace=True)
        df.sort_values('datetime', ascending=True, inplace=True, ignore_index=True)
        df = df[['datetime', 'open', 'high', 'low', 'close', 'volume']]  # Оставим только нужные столбцы
        df['volume'] = df['volume'].astype(float)  # Преобразуем тип данных
        df = df[df['volume'] != 0]  # Фильтруем ненужные строки
    else:
        print(f"Ошибка при выполнении запроса. Код статуса: {response.status_code}")
    return df


In [95]:
# Установка параметров для получения данных
intervals_map = {
        '5m': 5 * 60 * 1000,
        '15m': 15 * 60 * 1000,
        '30m': 30 * 60 * 1000,
        '1d': 24 * 60 * 60 * 1000
}
interval = '5m'
iterations = 24  # 24, 8, 4 на 5-, 15-, и 30-мин. интервале соответственно
end_time = datetime.datetime.now() # Устанавливаем значение на сегоднешнюю дату
start_time_0 = int((end_time - datetime.timedelta(minutes=iterations * intervals_map[interval] // (60 * 1000))).timestamp() * 1000)
frames = []

for i in range(iterations):
    df = get_data(start_time_0)
    frames.append(df)
    start_time_0 -= intervals_map[interval]  # Уменьшаем на 60000 миллисекунд для следующего запроса

data = pd.concat(frames, ignore_index=True)  # Объединяем все датафреймы в один
data.sort_values('datetime', ascending=True, inplace=True)
data.set_index('datetime', inplace=True)


In [96]:
# Уберем лишние данные. Оставим только столбцы open, close, high, low и volume
historical_data = data[['open', 'high', 'low', 'close', 'volume']]

# Проверяем данные на наличие пропусков и типов переменных
historical_data.info()
historical_data

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 852 entries, 2025-01-04 14:25:00 to 2025-01-04 18:15:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    852 non-null    object 
 1   high    852 non-null    object 
 2   low     852 non-null    object 
 3   close   852 non-null    object 
 4   volume  852 non-null    float64
dtypes: float64(1), object(4)
memory usage: 39.9+ KB


,open,high,low,close,volume
datetime,,,,,
2025-01-04 14:25:00,97959.99000000,98013.32000000,97945.15000000,98000.00000000,41.19545
2025-01-04 14:30:00,98000.01000000,98000.01000000,97940.64000000,97995.64000000,30.84397
2025-01-04 14:30:00,98000.01000000,98000.01000000,97940.64000000,97995.64000000,30.84397
2025-01-04 14:35:00,97995.64000000,98080.15000000,97995.64000000,98046.35000000,42.70695
2025-01-04 14:35:00,97995.64000000,98080.15000000,97995.64000000,98046.35000000,42.70695
...,...,...,...,...,...
2025-01-04 18:15:00,98065.13000000,98088.66000000,98042.19000000,98072.12000000,16.62437
2025-01-04 18:15:00,98065.13000000,98088.66000000,98042.19000000,98072.11000000,16.62538
2025-01-04 18:15:00,98065.13000000,98088.66000000,98042.19000000,98072.11000000,16.62538
